In [1]:
import os
import time
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import confusion_matrix, classification_report, f1_score
import seaborn as sns
import pandas as pd
import json

### ******** 1. SETUP ENVIRONMENT ********

In [2]:
# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Check device availability
print("PyTorch version:", torch.__version__)
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Using MPS device")
elif torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Using CUDA device: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("Using CPU device")


PyTorch version: 2.6.0
Using MPS device


### ******** 2. CONFIGURATION SETTINGS ********

In [ ]:
# Define paths
DATASET_PATH = "../../archive/images/images" 
RESULTS_DIR = "resnet50_baseline_kaggle_results"

# DATASET_PATH = "../../dataset-resized" 
# RESULTS_DIR = "resnet50_baseline_results"

# DATASET_PATH = "../../garbage-dataset-2"  
# RESULTS_DIR = "resnet50_baseline_garbage_dataset_results"

# Check if dataset path exists
if not os.path.exists(DATASET_PATH):
    raise FileNotFoundError(f"Dataset path {DATASET_PATH} does not exist")

# Create results directory
os.makedirs(RESULTS_DIR, exist_ok=True)

# Define baseline hyperparameters
LEARNING_RATE = 0.001
BATCH_SIZE = 32
OPTIMIZER = 'Adam'
EPOCHS = 30
DROPOUT_RATE = 0.5
UNFREEZE_LAYERS = 'none'
NUM_WORKERS = 4  

In [4]:
# ******** 3. DATA PREPARATION ********

In [5]:
def get_transforms():
    """Define image transformations for training and validation"""
    train_transform = transforms.Compose([
        transforms.Resize((240, 240)),
        transforms.CenterCrop((224, 224)),
        transforms.RandomHorizontalFlip(),  # Basic augmentation for better generalization
        transforms.RandomRotation(10),      # Minor rotation augmentation
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    
    val_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    
    return train_transform, val_transform

def load_data(batch_size=BATCH_SIZE, num_workers=NUM_WORKERS):
    """Load and prepare the dataset with train/val/test splits"""
    train_transform, val_transform = get_transforms()
    
    try:
        # Load the full dataset
        full_dataset = datasets.ImageFolder(DATASET_PATH, transform=train_transform)
        class_names = full_dataset.classes
        num_classes = len(class_names)
        
        # Calculate sizes for splits (70% train, 20% val, 10% test)
        total_size = len(full_dataset)
        train_size = int(0.7 * total_size)
        val_size = int(0.2 * total_size)
        test_size = total_size - train_size - val_size
        
        # Split the dataset
        train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(
            full_dataset, [train_size, val_size, test_size]
        )
        
        # Apply validation transform to validation and test sets
        val_dataset.dataset.transform = val_transform
        test_dataset.dataset.transform = val_transform
        
        # Create data loaders with optimized settings
        train_loader = DataLoader(
            train_dataset, 
            batch_size=batch_size, 
            shuffle=True, 
            num_workers=num_workers,
            pin_memory=True if device.type != 'cpu' else False
        )
        
        val_loader = DataLoader(
            val_dataset, 
            batch_size=batch_size, 
            shuffle=False, 
            num_workers=num_workers,
            pin_memory=True if device.type != 'cpu' else False
        )
        
        test_loader = DataLoader(
            test_dataset, 
            batch_size=batch_size, 
            shuffle=False, 
            num_workers=num_workers,
            pin_memory=True if device.type != 'cpu' else False
        )
        
        print(f"Dataset loaded: {len(train_dataset)} training, {len(val_dataset)} validation, {len(test_dataset)} test images")
        print(f"Classes: {class_names}")
        
        # Print dataset distribution
        class_counts = {class_name: 0 for class_name in class_names}
        for _, class_idx in full_dataset.samples:
            class_counts[class_names[class_idx]] += 1
            
        print("Class distribution:")
        for class_name, count in class_counts.items():
            print(f"  {class_name}: {count} images ({count/total_size:.1%})")
        
        return train_loader, val_loader, test_loader, class_names, num_classes
    
    except Exception as e:
        print(f"Error loading dataset: {e}")
        raise

### ******** 4. MODEL ARCHITECTURE ********

In [6]:

def create_resnet50(num_classes, dropout_rate=DROPOUT_RATE):
    """Create ResNet-50 model with pretrained weights"""
    try:
        # Load pretrained model
        model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)
        
        # Freeze all parameters for baseline model
        for param in model.parameters():
            param.requires_grad = False
        
        # Replace the fully connected layer
        num_features = model.fc.in_features
        model.fc = nn.Sequential(
            nn.Dropout(p=dropout_rate),
            nn.Linear(num_features, num_classes)
        )
        
        # Always unfreeze the fully connected layer
        for param in model.fc.parameters():
            param.requires_grad = True
        
        # If specified, unfreeze additional layers based on the strategy
        if UNFREEZE_LAYERS == 'last_block':
            # Unfreeze the last layer (layer4) for ResNet50
            for param in model.layer4.parameters():
                param.requires_grad = True
        elif UNFREEZE_LAYERS == 'all':
            # Unfreeze all layers
            for param in model.parameters():
                param.requires_grad = True
        
        return model
    
    except Exception as e:
        print(f"Error creating model: {e}")
        raise



### ******** 5. TRAINING SETUP ********

In [7]:
class EarlyStopping:
    """Early stopping to prevent overfitting"""
    def __init__(self, patience=7, verbose=True, path='best_model.pth', delta=0):
        self.patience = patience
        self.verbose = verbose
        self.path = path
        self.delta = delta
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = float('inf')
        
    def __call__(self, val_loss, model):
        score = -val_loss
        
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0
    
    def save_checkpoint(self, val_loss, model):
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}). Saving model...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

In [8]:
# ******** 6. TRAINING FUNCTION ********

In [9]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=EPOCHS, path='model.pth'):
    """Train the model and track performance metrics"""
    early_stopping = EarlyStopping(patience=7, path=path)
    
    # History tracking
    history = {
        'train_loss': [], 'val_loss': [], 
        'train_acc': [], 'val_acc': [], 
        'f1': []
    }
    
    model = model.to(device)
    start_time = time.time()
    
    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        
        # Training phase
        model.train()
        running_loss = 0.0
        running_corrects = 0
        total_samples = 0
        
        for inputs, labels in tqdm(train_loader, desc=f"Training"):
            inputs, labels = inputs.to(device), labels.to(device)
            batch_size = inputs.size(0)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)
            
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item() * batch_size
            running_corrects += torch.sum(preds == labels.data).item()
            total_samples += batch_size
        
        epoch_loss = running_loss / total_samples
        epoch_acc = running_corrects / total_samples
        history['train_loss'].append(epoch_loss)
        history['train_acc'].append(epoch_acc)
        
        print(f'Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
        
        # Validation phase
        model.eval()
        running_loss = 0.0
        running_corrects = 0
        total_samples = 0
        all_preds = []
        all_labels = []
        
        with torch.no_grad():
            for inputs, labels in tqdm(val_loader, desc=f"Validation"):
                inputs, labels = inputs.to(device), labels.to(device)
                batch_size = inputs.size(0)
                
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
                
                running_loss += loss.item() * batch_size
                running_corrects += torch.sum(preds == labels.data).item()
                total_samples += batch_size
                
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
        
        epoch_loss = running_loss / total_samples
        epoch_acc = running_corrects / total_samples
        f1 = f1_score(all_labels, all_preds, average='weighted')
        
        history['val_loss'].append(epoch_loss)
        history['val_acc'].append(epoch_acc)
        history['f1'].append(f1)
        
        print(f'Val Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f} F1: {f1:.4f}')
        
        # Early stopping check
        early_stopping(epoch_loss, model)
        if early_stopping.early_stop:
            print("Early stopping triggered")
            break
    
    # Calculate training time
    time_elapsed = time.time() - start_time
    print(f'Training completed in {time_elapsed//60:.0f}m {time_elapsed%60:.0f}s')
    
    # Load best model
    try:
        model.load_state_dict(torch.load(early_stopping.path))
        print(f"Loaded best model from {early_stopping.path}")
    except Exception as e:
        print(f"Warning: Could not load best model - {e}")
    
    return model, history

In [10]:
### ******** 7. EVALUATION FUNCTION ********

In [11]:
def evaluate_model(model, data_loader, class_names, save_path=None):
    """Evaluate the model and generate detailed metrics and visualizations"""
    model.eval()
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for inputs, labels in tqdm(data_loader, desc="Evaluating"):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    # Calculate metrics
    accuracy = np.mean(np.array(all_preds) == np.array(all_labels))
    cm = confusion_matrix(all_labels, all_preds)
    report = classification_report(all_labels, all_preds, target_names=class_names, output_dict=True)
    report_str = classification_report(all_labels, all_preds, target_names=class_names)
    
    # Print report
    print(f'Validation Accuracy: {accuracy:.4f}')
    print('\nClassification Report:')
    print(report_str)
    
    if save_path:
        # Save detailed report
        with open(f"{save_path}_report.txt", 'w') as f:
            f.write(f'Validation Accuracy: {accuracy:.4f}\n\n')
            f.write(report_str)
            
        # Save report as JSON for further analysis
        with open(f"{save_path}_report.json", 'w') as f:
            json.dump(report, f, indent=4)
        
        # Plot confusion matrix
        plt.figure(figsize=(10, 8))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
        plt.xlabel('Predicted')
        plt.ylabel('True')
        plt.title('Confusion Matrix')
        plt.tight_layout()
        plt.savefig(f"{save_path}_confusion_matrix.png")
        plt.close()
        
        # Plot normalized confusion matrix
        plt.figure(figsize=(10, 8))
        cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        sns.heatmap(cm_norm, annot=True, fmt='.2f', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
        plt.xlabel('Predicted')
        plt.ylabel('True')
        plt.title('Normalized Confusion Matrix')
        plt.tight_layout()
        plt.savefig(f"{save_path}_confusion_matrix_norm.png")
        plt.close()
        
        print(f'Evaluation results saved to {save_path}')
    
    return accuracy, report


### ******** 8. VISUALIZATION FUNCTION ********

In [12]:
def plot_history(history, save_path=None):
    """Plot training metrics history"""
    plt.figure(figsize=(15, 5))
    
    plt.subplot(1, 3, 1)
    plt.plot(history['train_loss'], label='Train Loss')
    plt.plot(history['val_loss'], label='Val Loss')
    plt.title('Loss')
    plt.xlabel('Epoch')
    plt.grid(True, alpha=0.3)
    plt.legend()
    
    plt.subplot(1, 3, 2)
    plt.plot(history['train_acc'], label='Train Acc')
    plt.plot(history['val_acc'], label='Val Acc')
    plt.title('Accuracy')
    plt.xlabel('Epoch')
    plt.grid(True, alpha=0.3)
    plt.legend()
    
    plt.subplot(1, 3, 3)
    plt.plot(history['f1'], label='F1 Score')
    plt.title('F1 Score')
    plt.xlabel('Epoch')
    plt.grid(True, alpha=0.3)
    plt.legend()
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(f"{save_path}_history.png")
        print(f'Training history plot saved to {save_path}_history.png')
    
    plt.close()



### ******** 9. MAIN EXECUTION FUNCTION ********

In [13]:

def main():
    print("\n" + "="*50)
    print("ResNet-50 Baseline Model for Recycling Material Classification")
    print("="*50)
    
    # Create directory for baseline results
    baseline_dir = os.path.join(RESULTS_DIR, "baseline")
    os.makedirs(baseline_dir, exist_ok=True)
    
    # Load data
    train_loader, val_loader, test_loader, class_names, num_classes = load_data(batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
    
    # Create model
    model = create_resnet50(num_classes, dropout_rate=DROPOUT_RATE)
    
    # Count parameters
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    
    print(f"Model created: ResNet-50 with {num_classes} output classes")
    print(f"Total parameters: {total_params:,}")
    print(f"Trainable parameters: {trainable_params:,} ({trainable_params/total_params:.2%})")
    print(f"Training strategy: {UNFREEZE_LAYERS if UNFREEZE_LAYERS != 'none' else 'Only fully connected layer trained, backbone frozen'}")
    
    # Setup loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    if OPTIMIZER == 'Adam':
        optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), 
                              lr=LEARNING_RATE)
    else:  # SGD as fallback
        optimizer = optim.SGD(filter(lambda p: p.requires_grad, model.parameters()),
                             lr=LEARNING_RATE, momentum=0.9)
    
    # Save configuration 
    config = {
        'model': 'ResNet-50',
        'learning_rate': LEARNING_RATE,
        'batch_size': BATCH_SIZE,
        'optimizer': OPTIMIZER,
        'epochs': EPOCHS,
        'dropout_rate': DROPOUT_RATE,
        'unfreeze_layers': UNFREEZE_LAYERS,
        'num_workers': NUM_WORKERS,
        'total_parameters': total_params,
        'trainable_parameters': trainable_params,
        'device': str(device),
        'dataset_path': DATASET_PATH,
        'num_classes': num_classes,
        'class_names': class_names
    }
    
    with open(os.path.join(baseline_dir, 'config.json'), 'w') as f:
        json.dump(config, f, indent=4)
    
    # Train the model
    print("\nStarting model training...")
    model_path = os.path.join(baseline_dir, 'best_model.pth')
    model, history = train_model(
        model, train_loader, val_loader, criterion, optimizer, 
        num_epochs=EPOCHS, path=model_path
    )
    
    # Plot and save training history
    history_path = os.path.join(baseline_dir, 'training_history')
    plot_history(history, save_path=history_path)
    
    # Evaluate on validation set
    print("\nEvaluating on validation set:")
    val_results_path = os.path.join(baseline_dir, 'validation_results')
    val_accuracy, val_report = evaluate_model(model, val_loader, class_names, save_path=val_results_path)
    
    # Evaluate on test set
    print("\nEvaluating on test set:")
    test_results_path = os.path.join(baseline_dir, 'test_results')
    test_accuracy, test_report = evaluate_model(model, test_loader, class_names, save_path=test_results_path)
    
    # Save model summary
    model_summary = {
        'model_type': 'ResNet-50',
        'num_classes': num_classes,
        'class_names': class_names,
        'parameters': total_params,
        'trainable_parameters': trainable_params,
        'val_accuracy': val_accuracy,
        'test_accuracy': test_accuracy,
        'val_f1_score': val_report['weighted avg']['f1-score'],
        'test_f1_score': test_report['weighted avg']['f1-score'],
        'per_class_f1': {cls: test_report[cls]['f1-score'] for cls in class_names}
    }
    
    with open(os.path.join(baseline_dir, 'model_summary.json'), 'w') as f:
        json.dump(model_summary, f, indent=4)
    
    print("\n" + "="*50)
    print("BASELINE MODEL SUMMARY")
    print("="*50)
    print(f"Model: ResNet-50")
    print(f"Classes: {class_names}")
    print(f"Total Parameters: {model_summary['parameters']:,}")
    print(f"Trainable Parameters: {model_summary['trainable_parameters']:,}")
    print(f"Validation Accuracy: {val_accuracy:.4f}")
    print(f"Test Accuracy: {test_accuracy:.4f}")
    print(f"Validation F1 Score: {val_report['weighted avg']['f1-score']:.4f}")
    print(f"Test F1 Score: {test_report['weighted avg']['f1-score']:.4f}")
    print("="*50)
    print(f"Full results saved to {baseline_dir}")
    
    return model, test_accuracy, test_report

if __name__ == "__main__":
    main()


ResNet-50 Baseline Model for Recycling Material Classification
Dataset loaded: 10500 training, 3000 validation, 1500 test images
Classes: ['aerosol_cans', 'aluminum_food_cans', 'aluminum_soda_cans', 'cardboard_boxes', 'cardboard_packaging', 'clothing', 'coffee_grounds', 'disposable_plastic_cutlery', 'eggshells', 'food_waste', 'glass_beverage_bottles', 'glass_cosmetic_containers', 'glass_food_jars', 'magazines', 'newspaper', 'office_paper', 'paper_cups', 'plastic_cup_lids', 'plastic_detergent_bottles', 'plastic_food_containers', 'plastic_shopping_bags', 'plastic_soda_bottles', 'plastic_straws', 'plastic_trash_bags', 'plastic_water_bottles', 'shoes', 'steel_food_cans', 'styrofoam_cups', 'styrofoam_food_containers', 'tea_bags']
Class distribution:
  aerosol_cans: 500 images (3.3%)
  aluminum_food_cans: 500 images (3.3%)
  aluminum_soda_cans: 500 images (3.3%)
  cardboard_boxes: 500 images (3.3%)
  cardboard_packaging: 500 images (3.3%)
  clothing: 500 images (3.3%)
  coffee_grounds: 500 

Training: 100%|███████████████████████████████| 329/329 [01:43<00:00,  3.19it/s]


Train Loss: 1.6008 Acc: 0.6188


Validation: 100%|███████████████████████████████| 94/94 [00:42<00:00,  2.20it/s]


Val Loss: 0.9564 Acc: 0.7497 F1: 0.7448
Validation loss decreased (inf --> 0.956367). Saving model...
Epoch 2/30


...

Val Loss: 0.5227 Acc: 0.8380 F1: 0.8367
EarlyStopping counter: 5 out of 7
Training completed in 71m 47s
Loaded best model from resnet50_baseline_kaggle_results/baseline/best_model.pth
Training history plot saved to resnet50_baseline_kaggle_results/baseline/training_history_history.png

Evaluating on validation set:


Evaluating: 100%|███████████████████████████████| 94/94 [00:42<00:00,  2.21it/s]


Validation Accuracy: 0.8370

Classification Report:
                            precision    recall  f1-score   support

              aerosol_cans       0.93      0.91      0.92       109
        aluminum_food_cans       0.45      0.46      0.45        96
        aluminum_soda_cans       0.82      0.81      0.82       107
           cardboard_boxes       0.57      0.68      0.62        92
       cardboard_packaging       0.62      0.56      0.59       106
                  clothing       0.86      0.92      0.89       102
            coffee_grounds       0.93      0.97      0.95       114
disposable_plastic_cutlery       0.98      0.99      0.98        84
                 eggshells       0.92      0.94      0.93        98
                food_waste       0.91      0.96      0.93        90
    glass_beverage_bottles       0.86      0.92      0.89       108
 glass_cosmetic_containers       0.88      0.87      0.87        91
           glass_food_jars       0.94      0.87      0.90      

Evaluating: 100%|███████████████████████████████| 47/47 [00:33<00:00,  1.41it/s]


Validation Accuracy: 0.8547

Classification Report:
                            precision    recall  f1-score   support

              aerosol_cans       0.88      0.95      0.91        55
        aluminum_food_cans       0.54      0.53      0.53        51
        aluminum_soda_cans       0.76      0.91      0.83        45
           cardboard_boxes       0.61      0.72      0.66        54
       cardboard_packaging       0.63      0.52      0.57        50
                  clothing       0.89      0.89      0.89        46
            coffee_grounds       0.90      0.98      0.94        53
disposable_plastic_cutlery       0.98      0.90      0.94        52
                 eggshells       0.94      0.94      0.94        47
                food_waste       0.98      1.00      0.99        45
    glass_beverage_bottles       0.86      0.94      0.90        54
 glass_cosmetic_containers       1.00      0.91      0.95        44
           glass_food_jars       0.90      0.84      0.87      